# SSVEP: Offline processing using Machine Leaning Method

## Step 0: Import neceessary toolboxes

In [1]:
# import require library for preprocess
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci
import scipy
import pickle
import seaborn as sns

# import require library for classification
from sklearn.svm import SVC # SVM library
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # LDA library
from sklearn.neighbors import KNeighborsClassifier # KNN library

from sklearn.metrics import classification_report,confusion_matrix # Result representation

In [2]:
select_feature = "fft"

## Step 1: Read data file

In [3]:
# read biosemi file (bdf)
raw = mne.io.read_raw_bdf("Testdata3.bdf", preload=True, verbose=False) 
eegbci.standardize(raw)

## Step 2: Data preprocessing -- set channel locations/ downsampling/ frequency filtering (bandpass)/ epoching

In [4]:
# Set channel location
montage = make_standard_montage("biosemi64")
raw.set_montage(montage, on_missing='ignore')

# Downsample data (from 1024 to 512Hz) to save storage space 
raw = raw.resample(512, verbose = False)

#Band pass and notch filter
raw = raw.copy().notch_filter(freqs=50)
raw = raw.copy().filter(l_freq=1, h_freq=40, verbose = False)

# Get events and timestamps
events = mne.find_events(raw, shortest_event = 0, verbose = False) 

# Create event dictionary 
event_dict =  {'12Hz': 8,
'24Hz': 4,
'6Hz': 10,
'30Hz': 2
}

# Use events and event dictionary to cut data into Epochs
ssvep_chans = ['O1','Oz','PO3','PO4','POz','Pz']  # Reject O2 becuase noisy channel
Epochs = mne.Epochs(raw, events, 
    tmin= -1.0,  
    tmax= 4.0,    
    event_id=event_dict,
    picks = ssvep_chans,
    preload = True,
    event_repeated='drop',
    baseline=(-1.0, 0.0),
    verbose=False
)

train_label = Epochs['12Hz','6Hz', '24Hz', '30Hz'].events[:,-1]


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    0.1s finished


## Step3: Feature extraction

## 3.1 Fast Fourier Transform

In [5]:
# Compute FFT for each epoch and return the power spectral density
def compute_fft(epoch_data, sampling_rate):

    num_epochs, num_channels, num_timepoints = epoch_data.shape

    freqs = np.fft.fftfreq(num_timepoints, 1 / sampling_rate)
    
    fft_data = np.zeros((num_epochs, num_channels, len(freqs)))

    # Compute FFT for each channel and each epoch
    for epoch_idx in range(num_epochs):
        for ch_idx in range(num_channels):        
            fft_result = scipy.fft.fft(epoch_data[epoch_idx, ch_idx, :])
            power_spectrum = np.abs(fft_result) ** 2  # Power = |FFT|^2
            fft_data[epoch_idx, ch_idx, :] = power_spectrum

    return fft_data, freqs

# Example usage
fft_out, freqs_out = compute_fft(Epochs.get_data(), 512)
print(np.shape(fft_out))
print(np.shape(freqs_out))

fft_train = np.stack([arr.flatten() for arr in fft_out])
print(fft_train.shape)


(20, 6, 2561)
(2561,)
(20, 15366)


C:\Users\pipo_\AppData\Local\Temp\ipykernel_23528\249329786.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  fft_out, freqs_out = compute_fft(Epochs.get_data(), 512)


## 3.2 Power Spectrum Density

In [6]:
psd_epoch = Epochs['12Hz','6Hz', '24Hz', '30Hz'].pick(ssvep_chans).compute_psd(fmin=1.0, fmax=40.0)
print(psd_epoch.shape)

psd_train = np.stack([arr.flatten() for arr in psd_epoch])
print(psd_train.shape)

    Using multitaper spectrum estimation with 7 DPSS windows
(20, 6, 195)
(20, 1170)


## Load test set

In [7]:
with open('SSVEP_test_epochs.pkl', 'rb') as f:
    test_epochs = pickle.load(f)

fft_out, freqs_out = compute_fft(test_epochs['12Hz','6Hz', '24Hz', '30Hz'].pick(ssvep_chans).get_data(), 512)
fft_test = np.stack([arr.flatten() for arr in fft_out])

psd_epoch = test_epochs['12Hz','6Hz', '24Hz', '30Hz'].pick(ssvep_chans).compute_psd(fmin=1.0, fmax=40.0)
psd_test = np.stack([arr.flatten() for arr in psd_epoch])

mapping = {2: 8, 4: 4, 8: 10, 10: 2}
test_label = np.vectorize(mapping.get)(test_epochs['12Hz','6Hz', '24Hz', '30Hz'].events[:,-1])


    Using multitaper spectrum estimation with 7 DPSS windows


C:\Users\pipo_\AppData\Local\Temp\ipykernel_23528\1931586479.py:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  fft_out, freqs_out = compute_fft(test_epochs['12Hz','6Hz', '24Hz', '30Hz'].pick(ssvep_chans).get_data(), 512)


## Step4: Classification

## 4.0 Select Feature as train and test set

In [8]:
if select_feature == "fft":
    x_train = fft_train
    x_test = fft_test

elif select_feature == 'psd':
    x_train = psd_train
    x_test = psd_test 

## 4.1 LDA

In [9]:
def GetConfusionMatrix(models, X_train, X_test, y_train, y_test, target_names):
    y_pred = models.predict(X_train)
    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred, target_names= target_names))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred = models.predict(X_test)
    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred, target_names= target_names))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))

    

In [10]:
y_train = train_label # Get true label
y_test = test_label

lda = LinearDiscriminantAnalysis(solver= "svd")
lda.fit(x_train, y_train)

print('accuracy', lda.score(x_train, y_train))
label_names = ['12Hz', '6Hz', '24Hz', '30Hz']

GetConfusionMatrix(lda, x_train, x_test, y_train, y_test, label_names)

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy 0.65
Classification TRAIN DATA 
              precision    recall  f1-score   support

        12Hz       0.50      0.80      0.62         5
         6Hz       0.50      0.40      0.44         5
        24Hz       0.75      0.60      0.67         5
        30Hz       1.00      0.80      0.89         5

    accuracy                           0.65        20
   macro avg       0.69      0.65      0.65        20
weighted avg       0.69      0.65      0.65        20

Confusion matrix 
[[4 1 0 0]
 [2 2 1 0]
 [1 1 3 0]
 [1 0 0 4]]
Classification TEST DATA 
              precision    recall  f1-score   support

        12Hz       1.00      0.17      0.29         6
         6Hz       0.29      0.40      0.33         5
        24Hz       0.07      0.20      0.11         5
        30Hz       0.00      0.00      0.00         6

    accuracy                           0.18        22
   macro avg       0.34      0.19      0.18        22
weighted avg       0.35      0.18      0.18        22



c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4.2 SVM

In [11]:
y_train = train_label # Get true label
y_test = test_label

svm_model = SVC(C = 1, kernel= 'rbf')  # Using a linear kernel
svm_model.fit(x_train, y_train)

print('accuracy', svm_model.score(x_train, y_train))
label_names = ['12Hz', '6Hz', '24Hz', '30Hz']

GetConfusionMatrix(svm_model, x_train, x_test, y_train, y_test, label_names)

accuracy 1.0
Classification TRAIN DATA 
              precision    recall  f1-score   support

        12Hz       1.00      1.00      1.00         5
         6Hz       1.00      1.00      1.00         5
        24Hz       1.00      1.00      1.00         5
        30Hz       1.00      1.00      1.00         5

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Confusion matrix 
[[5 0 0 0]
 [0 5 0 0]
 [0 0 5 0]
 [0 0 0 5]]
Classification TEST DATA 
              precision    recall  f1-score   support

        12Hz       0.80      0.67      0.73         6
         6Hz       0.45      1.00      0.62         5
        24Hz       1.00      1.00      1.00         5
        30Hz       1.00      0.17      0.29         6

    accuracy                           0.68        22
   macro avg       0.81      0.71      0.66        22
weighted avg       0.82      0.68      0.65        22

C

## 4.3 KNN

In [12]:
y_train = train_label # Get true label
y_test = test_label

knn = KNeighborsClassifier(n_neighbors= 10, weights = "distance")
knn.fit(x_train, y_train)

print('accuracy', knn.score(x_train, y_train))
label_names = ['12Hz', '6Hz', '24Hz', '30Hz']

GetConfusionMatrix(knn, x_train, x_test, y_train, y_test, label_names)

accuracy 1.0
Classification TRAIN DATA 
              precision    recall  f1-score   support

        12Hz       1.00      1.00      1.00         5
         6Hz       1.00      1.00      1.00         5
        24Hz       1.00      1.00      1.00         5
        30Hz       1.00      1.00      1.00         5

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Confusion matrix 
[[5 0 0 0]
 [0 5 0 0]
 [0 0 5 0]
 [0 0 0 5]]
Classification TEST DATA 
              precision    recall  f1-score   support

        12Hz       0.62      0.83      0.71         6
         6Hz       0.50      0.40      0.44         5
        24Hz       1.00      0.60      0.75         5
        30Hz       0.71      0.83      0.77         6

    accuracy                           0.68        22
   macro avg       0.71      0.67      0.67        22
weighted avg       0.71      0.68      0.68        22

C